In [0]:
import numpy as np 
import pandas as pd 
import cv2, pickle
from PIL import Image 
import pathlib
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.applications.vgg19 import VGG19
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [0]:
IMG_WIDTH=256
IMG_HEIGHT=256
BATCH_SIZE = 15

In [0]:
train = pickle.load(open("/content/drive/My Drive/pawan/dataset/train.npy",'rb'))
test = pickle.load(open("/content/drive/My Drive/pawan/dataset/test.npy",'rb'))
train = np.array(train)
test = np.array(test)
train_data = pd.read_csv("/content/drive/My Drive/pawan/dataset/train.csv")

encoder = LabelEncoder()
encoder.fit(train_data["target"])
encoded_Y = encoder.transform(train_data["target"])
y_labels = np_utils.to_categorical(encoded_Y)

In [8]:

from tensorflow.keras.models import Model
import tensorflow.keras as keras

resnet = VGG19(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3),pooling='max')

output = resnet.layers[-1].output
output = tf.keras.layers.Flatten()(output)
resnet = Model(resnet.input, output)

80142336/80134624 [==============================] - 1s 0us/step


In [0]:
for layer in resnet.layers:
    layer.trainable = False

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D, Dropout

num_classes = 8

model = Sequential()
model.add(resnet)
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model (Model)                (None, 512)               20024384  
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 8200      
Total params: 20,557,896
Trainable params: 533,512
Non-trainable params: 20,024,384
_________________________________________________________________


In [0]:
adam = tf.keras.optimizers.Adam(learning_rate=0.0001)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                              restore_best_weights=False
                                              )


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                   factor=0.3,
                                   patience=5,
                                   verbose=1,
                                   min_delta=1e-3,min_lr = 1e-7,
                                   )

In [0]:
model.compile(optimizer = adam, 
              loss = 'categorical_crossentropy', 
              metrics=['accuracy',tfa.metrics.F1Score(num_classes=num_classes)])

In [0]:
model.fit(train, y_labels,steps_per_epoch=np.ceil(float(train.shape[0]) / float(BATCH_SIZE)),
                        epochs = 50,callbacks=[early_stop,reduce_lr])

Epoch 1/50
25/25 [==============================] - 293s 12s/step - loss: 60.8457 - accuracy: 0.1731 - f1_score: 0.1667 - lr: 1.0000e-04
Epoch 2/50
25/25 [==============================] - 293s 12s/step - loss: 44.7519 - accuracy: 0.2830 - f1_score: 0.2821 - lr: 1.0000e-04
Epoch 3/50
25/25 [==============================] - 293s 12s/step - loss: 32.9891 - accuracy: 0.3681 - f1_score: 0.3611 - lr: 1.0000e-04
Epoch 4/50
25/25 [==============================] - 292s 12s/step - loss: 23.2821 - accuracy: 0.4808 - f1_score: 0.4709 - lr: 1.0000e-04
Epoch 5/50
25/25 [==============================] - 292s 12s/step - loss: 18.1577 - accuracy: 0.5522 - f1_score: 0.5544 - lr: 1.0000e-04
Epoch 6/50
25/25 [==============================] - 292s 12s/step - loss: 16.3903 - accuracy: 0.5577 - f1_score: 0.5565 - lr: 1.0000e-04
Epoch 7/50
25/25 [==============================] - 293s 12s/step - loss: 12.0782 - accuracy: 0.5907 - f1_score: 0.5941 - lr: 1.0000e-04
Epoch 8/50
25/25 [=======================